# Analyse de questionnaire : enquête sur les pratiques numériques des chercheurs

- comparer SHS/non SHS
- décrire le paysage (réseau)

### Chargement des bibliothèques

In [1]:
# Pour la manipulation de tableaux
import pandas as pd

# Pour la visualisation
import matplotlib.pyplot as plt

# Pour les statistiques
import pyshs

# Pour la classification
from sklearn.cluster import AgglomerativeClustering

# Enlever les warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv("./data/raw/SOSP_Export_base de données diffusable.csv")
meta = pd.read_csv("./data/raw/SOSP_metadonnees_variables.csv")

### 1.2 Mise en forme

La première étape est de regarder les données existantes (soit avec un tableur, soit directement dans le Notebook)

In [3]:
data.head()

,Id,contexte_travail,fonction_recherche,statut_professionnel,statut_professionnel_Autre,annee_premiere_publi,Systeme_exploitation,Usage_telephone_mobile,Outils_gestion_travail_av_confinement_visioconférence,Outils_gestion_travail_av_confinement_planification_RDV,...,Souhait_partage_donnees_produites_discipline,Souhait_partage_donnees_produites_pays_continent,Souhait_partage_donnees_produites_domaine_economique,Souhait_partage_donnees_produites_secteur_associatif,Souhait_partage_donnees_produites_sans_restriction,compatibilite_diffusion_partage,evolutions_pratiques_numeriques,sexe,disciplines_9niv,annee_de_naissance-recod10niv
0,1,nous sommes entre 6 et 10 personnes,professeur / professeure des universités et as...,fonctionnaire ou assimilé,NaN,1995,Windows,rarement,parfois,souvent,...,oui,oui,oui,oui,oui,oui,je ne sais pas,un homme,Sciences humaines,61 à 65 ans
1,2,nous sommes plus de 50 personnes,ingénieur / ingénieure de recherche,en CDI,NaN,2005,Windows,rarement,parfois,parfois,...,oui,oui,oui,oui,oui,NaN,NaN,un homme,Médecine,46 à 50 ans
2,3,nous sommes entre 2 et 5 personnes,maître / maîtresse de conférences et assimilés,fonctionnaire ou assimilé,NaN,2006,Windows,parfois,rarement,souvent,...,oui,oui,oui,oui,oui,"oui, tout est dans la chronologie",non,un homme,Médecine,36 à 40 ans
3,4,nous sommes entre 2 et 5 personnes,"directeur / directrice de recherche (CNRS, INS...",fonctionnaire ou assimilé,NaN,1989,Windows;autres OS (Android...),souvent,rarement,souvent,...,oui,oui,non,non,non,Oui,Aucune idée,une femme,Médecine,56 à 60 ans
4,5,nous sommes entre 2 et 5 personnes,doctorant / doctorante (y compris CIFRE),en CDD,NaN,2015,MacOS,souvent,rarement,parfois,...,oui,oui,oui,oui,oui,Oui,Oui,une femme,Médecine,31 à 35 ans


Regardons aussi les métadonnées.

In [4]:
questions = meta[31:]  # le tableau des questions ne commence qu'à la ligne 31
questions = questions[questions.columns[0:2]]  # garder les deux premières colonnes
questions.columns = ["variable", "question"]  # renommer les colonnes
questions = questions.set_index("variable")  # mettre le nom en index

# Recodage du genre

reco = {
    "un homme": "Homme",
    "une femme": "Femme",
    "je ne souhaite pas répondre": "Pas de réponse",
}
data["sexe"] = data["sexe"].replace(reco)

# Recodage de l'âge
reco = {
    "61 à 65 ans": "61-65",
    "46 à 50 ans": "46-50",
    "36 à 40 ans": "36-40",
    "56 à 60 ans": "56-60",
    "31 à 35 ans": "31-35",
    "25 à 30 ans": "25-30",
    "51 à 55 ans": "51-55",
    "41 à 45 ans": "41-45",
    "66 ans et plus": "66+",
    "moins de 25 ans": "18-25",
}
data["age"] = data["annee_de_naissance-recod10niv"].replace(reco)

# Recodage de la discipline
reco = {
    "Lettres et Arts": "SHS",
    "Sciences humaines": "SHS",
    "Sciences sociales": "SHS",
    "Physique, Sciences de la terre et de l'Univers": "Physique & Univers",
}
data["disciplines_reco"] = data["disciplines_9niv"].replace(reco)

# Passage de l'année de première publi en variable qualitative
data["annee_premiere_publi_quali"] = pd.cut(
    data["annee_premiere_publi"], range(1930, 2030, 10)
)

# Recodage du statut/fonction
reco = {
    "professeur / professeure des universités et assimilés": "EC/C cat A",
    "ingénieur / ingénieure de recherche": "Ingénieur",
    "maître / maîtresse de conférences et assimilés": "EC/C cat B",
    "directeur / directrice de recherche (CNRS, INSERM, INRAE, CNES...)": "EC/C cat A",
    "doctorant / doctorante (y compris CIFRE)": "Doctorant",
    "ingénieur / ingénieure d'études": "Ingénieur",
    "chargé / chargée de recherche (CNRS, INSERM, INRAE, CNES...)": "EC/C cat B",
    "chercheur / chercheure dans le secteur privé (ingénieur R&D, consultant, expert, chef de projet)": "Privé R&D",
    "responsable de service R&D dans le secteur privé": "Privé R&D",
}
data["fonction"] = data["fonction_recherche"].replace(reco)

reco = {
    "fonctionnaire ou assimilé": "fonctionnaire",
    "en CDI": "cdi",
    "en CDD": "cdd",
    "travailleur indépendant": "autre",
    "contrat doctoral": ":cdd",
    "retraité": "autre",
    "post-doc": "cdd",
    "doctorant non financé": "autre",
    "professeur / chercheur émérite": "autre",
    "chômage": "autre",
    "autre": "autre",
    "vacataire": "autre",
}
data["statut"] = data["statut_professionnel"].replace(reco)

# Normalisation des recodage avec un ordre en regroupant des modalités
reco = {
    "rarement": "2-Parfois",
    "parfois": "2-Parfois",
    "souvent": "1-Souvent",
    "jamais": "3-Jamais",
    "toujours": "1-Souvent", # choix de réunir toujours et souvent
}

data["telephone_mobile"] = data["Usage_telephone_mobile"].replace(reco)
data["donnees_numeriques"] = data["type_donnees_nombres_valeurs_numeriques"].replace(
    reco
)
data["donnees_textes"] = data["type_donnees_textes"].replace(reco)
data["donnees_images"] = data["type_donnees_images"].replace(reco)
data["donnees_videos"] = data["type_donnees_videos"].replace(reco)
data["donnees_3D"] = data["type_donnees_ 3D"].replace(reco)
data["donnees_sons"] = data["type_donnees_sons"].replace(reco)
data["donnees_reutilisation"] = data["Reutilisation_donnees"].replace(reco)
data["usage_outils_nettoyage"] = data["Usage_outils_nettoyage"].replace(reco)
data["usage_outils_analyse"] = data["Usage_outils_analyse"].replace(reco)
data["usage_outils_visualisation"] = data["Usage_outils_visualisation"].replace(reco)
data["usage_outils_qualification"] = data[
    "usage_outils_systemes_qualif_donnees"
].replace(reco)
data["usage_outils_traitement_texte"] = data[
    "Outils_redaction_documents_traitement_texte"
].replace(reco)
data["usage_outils_traitement_texte_ligne"] = data[
    "Outils_redaction_documents_traitement_texte_ligne"
].replace(reco)
data["usage_outils_latex"] = data["Outils_redaction_documents_editeur_Latex"].replace(
    reco
)
data["usage_outils_balisage"] = data["Outils_redaction_documents_balisage"].replace(
    reco
)
data["usage_outils_aide_redaction"] = data[
    "Outils_redaction_documents_aide_redaction"
].replace(reco)
data["usage_outils_aide_redaction_specifique"] = data[
    "Outils_redaction_documents_formats_specifiques"
].replace(reco)
data["usage_outils_references"] = data[
    "Outils_redaction_documents_gestionnaire_references"
].replace(reco)
data["usage_outils_traduction"] = data["Outils_redaction_documents_traduction"].replace(
    reco
)
data["usage_outils_pao"] = data["Outils_redaction_documents_PAO"].replace(reco)
data["usage_outils_figure"] = data["outils_production_figure"].replace(reco)

# recodage d'une colonne de choix multiples
reponses = {
    "des logiciels créés à cette fin (seul ou avec des collègues)": "logiciels_dedies",
    "des logiciels gratuits avec des fonctionnalités premium": "logiciels_freemium",
    "des logiciels internes à mon organisation": "logiciels_internes",
    "des logiciels issus d'un projet de recherche": "logiciels_projet",
    "des logiciels libres et gratuits": "logiciels_libres",
    "des logiciels partiellement ou complètement gratuits mais propriétaires": "logiciels_proprietaires",
    "des logiciels payants": "logiciels_payants",
    'des logiciels payants "crackés"': "logiciels_crackes",
    "des logiciels version d'essai ou démo": "logiciels_demo",
}
# Pour chaque ligne regarder si la réponse est présente (Vraie) ou absente (Fausse)
for i in reponses:
    data[reponses[i]] = (
        data["types_logiciels_production_donnees"]
        .str.contains(i)
        .fillna(False)
        .apply(str)
    )

# Recodage des systèmes d'exploitation
data["OS_windows"] = data["Systeme_exploitation"].str.contains("Windows").apply(str)
data["OS_linux"] = data["Systeme_exploitation"].str.contains("Linux|Unix").apply(str)
data["OS_mac"] = data["Systeme_exploitation"].str.contains("Mac").apply(str)
data["OS_multiples"] = data["Systeme_exploitation"].str.contains(";").apply(str)

**Remarque** : nous réutilisons des données, elles ne sont pas parfaitement adaptées à ce que nous voudrions faire. Ainsi, la fonction/statut professionnel ne semble pas découler d'une grille très cohérente (être postdoc est un CDD par exemple, et les deux modalités existent). Certains recodages correctifs pourraient être envisagés.

## Analyse des données non structurées du questionnaire

In [6]:
var_libre = [
    "logiciel_production_donnees",
    "outils_nettoyage",
    "Outils_analyses",
    "Outils_visualisation",
    "outils_production_figure",
]

# Créer une nouvelle colonne qui agrège tous les textes
data["outils"] = data[var_libre].apply(
    lambda l: "|".join([u for u in l if pd.notnull(u)]), axis=1
)

data[var_libre]

,logiciel_production_donnees,outils_nettoyage,Outils_analyses,Outils_visualisation,outils_production_figure
0,Lime survey,NaN,tableur pour les données statistiques,NaN,aucun
1,excell word,clean,outil statistique,NaN,NaN
2,JASP,NaN,NaN,NaN,ppt
3,"Question peu claire. Excel, SPSS, logiciels d...",NaN,"Excel, SPSS",NaN,NaN
4,?,NaN,Excel r,Image j dti studii,Ppt image j
...,...,...,...,...,...
1084,NaN,NaN,EpiInfo\nXlstat,EpiInfo\nXlstat\nExcel,Excel\nPowerpoint\nThe Gimp\nGoogle map
1085,NaN,NaN,NaN,"Xpdf, Evince",NaN
1086,NaN,"openrefine, talend",NaN,NaN,NaN
1087,NaN,NaN,NaN,NaN,"XFig .... mais c'est dépassé, je devrais changer"


In [7]:
questions.loc[var_libre]

,question
variable,
logiciel_production_donnees,Quels logiciels utilisez-vous pour produire de...
outils_nettoyage,Pouvez-vous précisez lesquels ?
Outils_analyses,Pouvez-vous précisez lesquels ?
Outils_visualisation,Pouvez-vous précisez lesquels ?
outils_production_figure,Quels outils utilisez-vous pour générer des fi...


### 4.1 Extraire les termes des réponses

Extraire tous les termes (avec la limite que des logiciels comme R sont représentés par une lettre, ou Image J en deux mots... un travail plus précis devrait identifier ces bigrammes. De nombreuses ressources permettent d'aller plus loin, par exemple : https://towardsdatascience.com/text-analysis-basics-in-python-443282942ec5)

In [8]:
# Définir une fonction qui décomposer une phrase en mots en enlevant les mots trop fréquents
def tokenizer(ligne):
    ponctuation = [".", ",", ";", "-", "\n", "(", ")", "'",":","?","|"]
    stopwords = [
        "de",
        "ou",
        "et",
        "des",
        "en",
        "d",
        "je",
        "par",
        "pour",
        "à",
        "logiciels",
        "pas",
        "la",
        "les",
        "données",
        "logiciel",
        "/","le","sur","un","une"
    ]
    for i in ponctuation:
        ligne = ligne.replace(i, " ")
    return [i for i in ligne.lower().split(" ") if len(i) > 0 and not i in stopwords]


# Application sur la colonne
data["outils_token"] = data["outils"].apply(tokenizer)

# Construction d'un compteur
from collections import Counter

# Comptage des mots des phrases décomposés
compteur = Counter([j for i in list(data["outils_token"]) for j in i])

# Afficher les termes les plus communs
compteur.most_common(20)

[('excel', 675),
 ('r', 641),
 ('python', 417),
 ('matlab', 268),
 ('word', 214),
 ('qgis', 156),
 ('powerpoint', 154),
 ('office', 124),
 ('gimp', 119),
 ('inkscape', 118),
 ('illustrator', 116),
 ('adobe', 113),
 ('outils', 113),
 ('photoshop', 90),
 ('spss', 87),
 ('imagej', 83),
 ('origin', 79),
 ('sas', 74),
 ('gnuplot', 68),
 ('latex', 64)]

Créer un dictionnaire de recodage

In [10]:
data["outils"].apply(lambda x : x.replace("\n"," ")).str.lower().to_csv("./outils_shs.csv")

Recharger le dictionnaire de recodage

In [13]:
data_reco = pd.read_csv("./outils_shs_reco.csv")

In [13]:
len(data_reco)

492

On regroupe word ; excel ; office ; tableur ; traitement de texte. Pour cela on crée une fonction

In [11]:
def reco(x):
    x = x.strip()
    if x in ["excel","word","office","libreoffice","openoffice","powerpoint","open office",
             "libre office","calc","traitement de texte","tableur"]:
        return "Office-word,calc&co"
    return x

In [14]:
d = [i.split("|") for i in list(data_reco["outils"].dropna())]
#d = [j.strip() for i in d for j in i]

In [15]:
import networkx as nx
from itertools import combinations
from ipysigma import Sigma

In [16]:
(pd.Series([j for i in d for j in i]).value_counts()/len(data_reco))[0:10]

excel          0.380081
word           0.317073
r              0.235772
qgis           0.123984
office         0.107724
powerpoint     0.101626
spss           0.093496
photoshop      0.077236
illustrator    0.075203
python         0.056911
Name: count, dtype: float64

In [17]:
network = nx.Graph()

for l in d:
    l = [reco(u) for u in l if pd.notnull(reco(u)) and reco(u)!=""]
    for e in l:
        if not e in network.nodes:
            network.add_node(e, label=e, weight=1)
        else:
            network.nodes[e]["weight"] += 1
            
    for i, j in combinations(l, 2): 
        if not network.has_edge(i, j):
            network.add_edge(i, j, weight=1)
        else:
            network[i][j]["weight"] += 1

Sortir vers un format utilisable sur Gephi

In [9]:
nx.write_graphml(network,"reseau.graphml")

Visualisation

In [18]:
#network.remove_node("bureautique")
Sigma(network,node_size="weight")

Sigma(nx.Graph with 424 nodes and 2,559 edges)

In [21]:
network2 = network.copy()
network2.remove_node("Office-word,calc&co")
Sigma(network2,node_size="weight")

Sigma(nx.Graph with 423 nodes and 2,232 edges)

In [22]:
len(compteur)

2824

In [23]:
f = data["disciplines_reco"]=="SHS"
compteur_shs = Counter([j for i in list(data[f]["outils_token"]) for j in i])
print(len(compteur_shs))
compteur_shs.most_common(50)

1418


[('excel', 281),
 ('r', 245),
 ('word', 174),
 ('qgis', 100),
 ('adobe', 75),
 ('spss', 74),
 ('illustrator', 73),
 ('office', 64),
 ('photoshop', 56),
 ('python', 55),
 ('stata', 51),
 ('arcgis', 41),
 ('sig', 39),
 ('powerpoint', 39),
 ('traitement', 37),
 ('iramuteq', 37),
 ('nvivo', 36),
 ('outils', 35),
 ('microsoft', 33),
 ('ne', 33),
 ('gephi', 32),
 ('libreoffice', 32),
 ('aucun', 31),
 ('suite', 31),
 ('sas', 31),
 ('texte', 28),
 ('sphinx', 28),
 ('gimp', 28),
 ('avec', 26),
 ('analyse', 25),
 ('inkscape', 24),
 ('j', 23),
 ('filemaker', 22),
 ('open', 21),
 ('limesurvey', 21),
 ('matlab', 20),
 ('statistiques', 19),
 ('access', 19),
 ('dans', 19),
 ('alceste', 18),
 ('txm', 18),
 ('sonal', 18),
 ('rstudio', 18),
 ('l', 17),
 ('tableur', 16),
 ('calc', 16),
 ('paint', 16),
 ('google', 16),
 ('question', 16),
 ('que', 16)]

In [23]:
len(compteur_shs)

1418

En modifiant itérativement la liste des stopwords, on peut améliorer l'identification des logiciels. Cette analyse dépasse l'objectif de ce notebook, et ouvre la voie à un traitement du langage naturel.

### 4.2 Usagers de Python et profils

Nous allons pour finir nous concentrer sur les utilisateurs de Python.

In [24]:
data["usage_python"] = data["outils_token"].apply(lambda x: "python" in x)

Il est alors possible de regarder les propriétés de ces utilisateurs de Python par rapport aux autres variables, comme les profils identifiés.

In [25]:
pyshs.tableau_croise(data, "clusters", "usage_python")

Clairement, il y a peu d'utilisateurs de Python dans la catégorie 1, et ils sont presque la moitié dans la catégorie 2 à utiliser ce type d'outils. Il est possible aussi de se poser la question de l'usage par rapport aux disciplines : 

In [26]:
pyshs.tableau_croise(data, "disciplines_reco", "usage_python")

usage_python,False,True,Total
disciplines_reco,,,
"Chimie, Matériaux",55 (93.2%),4 (6.8%),59 (100%)
"Mathématiques, Informatiques",93 (67.4%),45 (32.6%),138 (100%)
Médecine,48 (88.9%),6 (11.1%),54 (100%)
Physique & Univers,55 (53.4%),48 (46.6%),103 (100%)
SHS,466 (94.5%),27 (5.5%),493 (100%)
Science de l'ingénieur,58 (63.7%),33 (36.3%),91 (100%)
Sciences du vivant,125 (90.6%),13 (9.4%),138 (100%)
Total,900 (83.6%),176 (16.4%),1076 (100%)


Clairement, les physiciens sont les plus coutumiers de l'usage de Python ; en SHS l'usage est encore à ses débuts. Mais il ne demande qu'à progresser !

## Conclusion



<div class="alert alert-block alert-info">
    
Ce Notebook présente la démarche de réanalyse de données collectées lors d'une enquête par questionnaire pour étudier les pratiques d'outils numériques des chercheurs.<br>

Il retrace les différentes étapes nécessaires d'une analyse : chargement des données, exploration, puis recodage pour formuler une question de recherche précise, et enfin la mise en oeuvre d'une analyse spécifique. Dans notre cas, il s'agit de la construction de profils de chercheurs par rapport aux réponses qu'ils ont donné.<br>
    
Pour cela il présente différentes stratégies pratiques de préparation des données, permettant de garder la trace de l'ensemble des modifications réalisées sur les données brutes.<br>
    
La construction de ces catégories reposent sur une stratégie de clusterisation relevant du machine learning. Dans ce cas, c'est une classification hiérarchique ascendante réalisé après une ACM qui permet d'identifier trois catégories.<br>
    
Les profils identifiés sont ensuite interprétés en les rapportant aux variables utilisées pour la classification, et aux variables indépendantes.<br>
    
En complément de cette analyse, il pourrait être intéressant de confronter cette typologie à d'autres regroupements (par exemple avec 4 ou 5 classes) afin de vérifier la robustesse.
</div>

Aller plus loin : pour dépasser l'objectif descriptif de l'enquête, il pourrait être intéressant de prédire l'usage de Python en fonction du profil des répondants.